In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 945.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 13:06:58.593922


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- C1 = 49,000 (revenue coefficient for X1)
- C2 = 30 (revenue coefficient for X2)
- C3 = 150 (cost per hectolitre of syrup)
- H = 40 (number of 1.5 l COLI units produced per hectolitre of syrup)
- K = 20 (number of 1.5 l PEPSA units produced per hectolitre of syrup)
- P = 7,100 (net capacity of S1 production line in 1.5 l product units per hour)
- T = 40 (weekly working hours)
- W = 5 (number of working days per week)

Decision variables:
- X1 (number of 1.5 l COLI units produced)
- X2 (number of 1.5 l PEPSA units produced)

Objective function:
The objective is to maximize daily profit, which is given by the revenue from selling X1 and X2 units minus the cost of producing them. The revenue function is given as:

49,000X1 - X1^2 + 30X2 - 2X2^2

The cost of producing X1 and X2 units is the cost of syrup required to make them. Since each hectolitre of syrup produces 40 units of COLI and 20 units of PEPSA, the total number of hectolitres of syrup needed is:

(X1/40) + (X2/20)

Therefore, the cost of producing X1 and X2 units is:

150 * [(X1/40) + (X2/20)]

The daily profit can be expressed as:

Profit = 49,000X1 - X1^2 + 30X2 - 2X2^2 - 150 * [(X1/40) + (X2/20)]

Constraints:
1. Capacity constraint: The total number of 1.5 l product units produced per day cannot exceed the daily capacity of the S1 production line. The capacity of the S1 line is 7,100 units per hour, and the firm works 8 hours per day.

X1 + X2 ≤ 7100 * 8

2. Minimum production constraint: The firm is committed to produce at least half the amount of PEPSA than COLI.

X2 ≥ 0.5X1

3. Non-negativity constraints: The number of units produced must be non-negative.

X1 ≥ 0
X2 ≥ 0

Putting it all together, the mathematical optimization model is:

Maximize:
Profit = 49,000X1 - X1^2 + 30X2 - 2X2^2 - 150 * [(X1/40) + (X2/20)]

Subject to:
1. X1 + X2 ≤ 56,800
2. X2 ≥ 0.5X1
3. X1 ≥ 0
4. X2 ≥ 0

This model can be solved using linear or nonlinear optimization techniques to determine the optimal values of X1 and X2 that maximize the firm's daily profit while satisfying the given constraints.

In [7]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- C1 = 49,000 (revenue coefficient for X1)
- C2 = 30 (revenue coefficient for X2)
- C3 = 150 (cost per hectolitre of syrup)
- H = 40 (number of 1.5 l COLI units produced per hectolitre of syrup)
- K = 20 (number of 1.5 l PEPSA units produced per hectolitre of syrup)
- P = 7,100 (net capacity of S1 production line in 1.5 l product units per hour)
- T = 40 (weekly working hours)
- W = 5 (number of working days per week)

Decision variables:
- X1 (number of 1.5 l COLI units produced)
- X2 (number of 1.5 l PEPSA units produced)

Objective function:
The objective is to maximize daily profit, which is given by the revenue from selling X1 and X2 units minus the cost of producing them. The revenue function is given as:

49,000X1 - X1^2 + 30X2 - 2X2^2

The cost of producing X1 and X2 units is the cost of syrup required t

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters
model.C1 = 49000
model.C2 = 30
model.C3 = 150
model.H = 40
model.K = 20
model.P = 7100
model.T = 40
model.W = 5

# Define the decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# Define the objective function
model.obj = Objective(
    expr=model.C1 * model.X1 - model.X1**2 + model.C2 * model.X2 - 2 * model.X2**2
    - model.C3 * ((model.X1 / model.H) + (model.X2 / model.K)),
    sense=maximize,
)

# Define the constraints
model.capacity_constraint = Constraint(
    expr=model.X1 + model.X2 <= model.P * model.T * model.W
)

model.min_production_constraint = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('glpk')  # Use GLPK as the solver
results = solver.solve(model)  # Solve the model

# Print the results
print("Objective value:", model.obj())
print("X1:", model.X1())
print("X2:", model.X2())
```

Please note that this code uses the GLPK solver, which is a free and open-source solver for linear programming. If you want to use a different solver, you will need to replace 'glpk' with the name of the solver you want to use.

Also, this code assumes that the parameters are constant. If the parameters can change, you will need to modify the code to accept the parameters as input.

Finally, this code does not include any error handling. In a production environment, you would want to add error handling to deal with situations where the solver fails to find a solution, or where the input data is invalid.

In [11]:
print(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters
model.C1 = 49000
model.C2 = 30
model.C3 = 150
model.H = 40
model.K = 20
model.P = 7100
model.T = 40
model.W = 5

# Define the decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# Define the objective function
model.obj = Objective(
    expr=model.C1 * model.X1 - model.X1**2 + model.C2 * model.X2 - 2 * model.X2**2
    - model.C3 * ((model.X1 / model.H) + (model.X2 / model.K)),
    sense=maximize,
)

# Define the constraints
model.capacity_constraint = Constraint(
    expr=model.X1 + model.X2 <= model.P * model.T * model.W
)

model.min_production_constraint = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('glpk')  # Use GLPK as the solver
results = solver.solve(model)  # Solve the model

# Print the results
print("Objective val

### Code Executability

In [12]:
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters
model.C1 = 49000
model.C2 = 30
model.C3 = 150
model.H = 40
model.K = 20
model.P = 7100
model.T = 40
model.W = 5

# Define the decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# Define the objective function
model.obj = Objective(
    expr=model.C1 * model.X1 - model.X1**2 + model.C2 * model.X2 - 2 * model.X2**2
    - model.C3 * ((model.X1 / model.H) + (model.X2 / model.K)),
    sense=maximize,
)

# Define the constraints
model.capacity_constraint = Constraint(
    expr=model.X1 + model.X2 <= model.P * model.T * model.W
)

model.min_production_constraint = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')  # Use GLPK as the solver
results = solver.solve(model)  # Solve the model

# Print the results
print("Objective value:", model.obj())
print("X1:", model.X1())
print("X2:", model.X2())

Objective value: 400289176.04199195
X1: 16335.833333340024
X2: 8167.9166666600495


### Solution Correctness